In [26]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import operator
from collections import Counter
from operator import itemgetter
import time as time
import string

In [2]:
%%time
df_small = pd.read_csv('../data/small_sample.csv')
df = pd.read_csv('../data/large_sample.csv')
df_small.drop(columns=['Unnamed: 0'], inplace=True)
df.drop(columns=['Unnamed: 0'], inplace=True)
df.rating = np.ceil(df.rating)
df_small.rating = np.ceil(df_small.rating)

CPU times: user 4.94 s, sys: 540 ms, total: 5.48 s
Wall time: 5.94 s


In [3]:
def create_count(comment):
    for x in comment.split(' '):
        if x in di:
            di[x.lower()] += 1
        else:
            di[x.lower()] = 1
    return di

In [4]:
%%time
di = dict()
df.loc[df.rating == 1].loc[df.comment.apply(lambda x: len(x.split(' ')))<30].comment.apply(create_count)
one_dict = di
di = dict()
df.loc[df.rating == 2].loc[df.comment.apply(lambda x: len(x.split(' ')))<30].comment.apply(create_count)
two_dict = di
di = dict()
df.loc[df.rating == 3].loc[df.comment.apply(lambda x: len(x.split(' ')))<30].comment.apply(create_count)
three_dict = di
di = dict()
df.loc[df.rating == 4].loc[df.comment.apply(lambda x: len(x.split(' ')))<30].comment.apply(create_count)
four_dict = di
di = dict()
df.loc[df.rating == 5].loc[df.comment.apply(lambda x: len(x.split(' ')))<30].comment.apply(create_count)
five_dict = di
di = dict()
df.loc[df.rating == 6].loc[df.comment.apply(lambda x: len(x.split(' ')))<30].comment.apply(create_count)
six_dict = di
di = dict()
df.loc[df.rating == 7].loc[df.comment.apply(lambda x: len(x.split(' ')))<30].comment.apply(create_count)
seven_dict = di
di = dict()
df.loc[df.rating == 8].loc[df.comment.apply(lambda x: len(x.split(' ')))<30].comment.apply(create_count)
eight_dict = di
di = dict()
df.loc[df.rating == 9].loc[df.comment.apply(lambda x: len(x.split(' ')))<30].comment.apply(create_count)
nine_dict = di
di = dict()
df.loc[df.rating == 10].loc[df.comment.apply(lambda x: len(x.split(' ')))<30].comment.apply(create_count)
ten_dict = di
time = '37s'

CPU times: user 55.1 s, sys: 549 ms, total: 55.6 s
Wall time: 59.1 s


In [5]:
%%time
one_counter = Counter(one_dict)
two_counter = Counter(two_dict)
three_counter = Counter(three_dict)
four_counter = Counter(four_dict)
five_counter = Counter(five_dict)
six_counter = Counter(six_dict)
seven_counter = Counter(seven_dict)
eight_counter = Counter(eight_dict)
nine_counter = Counter(nine_dict)
ten_counter = Counter(ten_dict)

CPU times: user 36.4 ms, sys: 7.34 ms, total: 43.8 ms
Wall time: 74.8 ms


In [6]:
all_counter = one_counter + two_counter + three_counter + four_counter + five_counter + six_counter + seven_counter + eight_counter + nine_counter + ten_counter

In [7]:
all_dict = dict(Counter(all_counter))

In [8]:
%%time
new_data = {k: v for k, v in all_dict.items() if v > 5}

CPU times: user 30 ms, sys: 1.21 ms, total: 31.2 ms
Wall time: 31.3 ms


In [9]:
%%time
sortedList=sorted(new_data.values())
sorted_dict = dict()
for sortedKey in sortedList:
    for key, value in new_data.items():
        if value==sortedKey:
            sorted_dict[key]=value
            
time = '1min 32sec'

CPU times: user 2min 11s, sys: 150 ms, total: 2min 12s
Wall time: 2min 12s


In [10]:
%%time
dict_list = []
for key, value in sorted_dict.items():
    temp = [key,value]
    dict_list.append(temp)

CPU times: user 40.2 ms, sys: 2.65 ms, total: 42.8 ms
Wall time: 70.1 ms


In [11]:
testing_df = df.loc[df.comment.apply(lambda x: len(x.split(' ')))<30].copy()

In [12]:
def make_it_lower(comment):
    new_comment = []
    for x in comment.split(' '):
        new_comment.append(x.lower())
    return ' '.join(new_comment)

In [13]:
%%time
testing_df['lower_comment'] = testing_df.comment.apply(lambda x: make_it_lower(x))
testing_df.drop(columns=['comment'], inplace=True)
testing_df.rename(columns={'lower_comment':'comment'}, inplace=True)

CPU times: user 4.54 s, sys: 123 ms, total: 4.66 s
Wall time: 4.67 s


In [14]:
def word_count(comment, word, number):
    if word in comment.split(' '):
        if word in number:
            number[word] += comment.split(' ').count(word)
        else:
            number[word] = comment.split(' ').count(word)

In [22]:
%%time
def make_report(w, z, dictionary, dataframe):
    dict_list = dictionary
    testing_df = dataframe.copy()
    labels = ['word', 'average/std', 'percent_average', 'percent_std', 'rating', 'word_count', 'divisor', 'percent', 'time']
    to_return = pd.DataFrame(columns=labels)
    for word in dict_list[int(w):int(z)]:
        start_time = time.time()
        to_append = []
        std = 0
        average = 0
        for y in range(1,11):
            number = dict()
            testing_df.loc[testing_df.rating == y].comment.apply(word_count, word=word[0], number=number)
            divisor = testing_df.loc[df.rating == y].comment.count()
            if word[0] in number:
                percent = number[word[0]]/divisor
            else:
                percent = 0
                number[word[0]] = 0
            to_append.append([word[0], None, None, None, y, number[word[0]], divisor, percent, None])
        for_array = []
        for value in to_append:
            for_array.append(value[7])
        x = np.array(for_array)
        array1 = [[word[0], (np.sum(x)/10)/(np.std(x)), np.sum(x)/10, np.std(x), None, None, None, None, time.time()-start_time]]
        for x in to_append:
            array1.append(x)
        inbetween_df = pd.DataFrame.from_records(array1, columns=labels)
        to_return = pd.concat([to_return, inbetween_df])
    return to_return

CPU times: user 11 µs, sys: 1 µs, total: 12 µs
Wall time: 18.1 µs


In [23]:
def write_csv(low, high, df):
    df.to_csv('report_for_{}_to_{}.csv'.format(low, high))

In [24]:
def loop_write_report(dictionary, dataframe, number, count):
    num = int(number)
    size = int(count)
    first = 0
    while first < num - 1:
        if first + size < num - 2:
            report = make_report(first, first + size, dictionary, dataframe)
            write_csv(first, first + size, report)
            print('dataframe written from {} to {}'.format(first, first + size))
            first = first + size + 1
        else:
            report = make_report(first, num - 1, dictionary, dataframe)
            write_csv(first, num - 1, report)
            print('dataframe written from {} to {}'.format(first, num - 1))
            print('last dataframe')
            break

In [16]:
list_to_pass = dict_list

In [17]:
df = pd.read_csv('report_for_8040_to_8240.csv')

In [19]:
'(family' in list_to_pass

False

In [17]:
list_to_pass[8239]

['(family', 10]

In [18]:
list_to_pass[8240]

['adventures,', 10]

In [19]:
list_to_pass[8240:]

[['adventures,', 10],
 ['unexciting.', 10],
 ['rolling...', 10],
 ['incoming', 10],
 ['he/she', 10],
 ['owned:', 10],
 ['benefits.', 10],
 ['timers.', 10],
 ['(our', 10],
 ['dinosaurs', 10],
 ['politics,', 10],
 ['ha.', 10],
 ['bashing.', 10],
 ['comics', 10],
 ['fortune', 10],
 ['practicing', 10],
 ['frustration,', 10],
 ['model,', 10],
 ['bazillion', 10],
 ['novelty,', 10],
 ['him!', 10],
 ['hearts,', 10],
 ['(30min)', 10],
 ['reskin', 10],
 ['viking', 10],
 ['dependant,', 10],
 ['chart', 10],
 ['interaction;', 10],
 ['future!', 10],
 ['not).', 10],
 ["'go", 10],
 ['copies)', 10],
 ['speculation.', 10],
 ['biased.', 10],
 ['cave', 10],
 ['escalation', 10],
 ['age!', 10],
 ['pics', 10],
 ['revolution', 10],
 ['eludes', 10],
 ['great?', 10],
 ['seal', 10],
 ['dividing', 10],
 ['analyzed', 10],
 ['commands', 10],
 ['towers', 10],
 ['luck;', 10],
 ['uninterested', 10],
 ['homework.', 10],
 ['facts', 10],
 ['bloated.', 10],
 ['bumping', 10],
 ['innovation,', 10],
 ['sunday', 10],
 ['marin

In [20]:
new_list_to_pass = list_to_pass[8240:]

In [27]:
loop_write_report(dictionary=new_list_to_pass, dataframe=testing_df, number=len(dict_list), count=200)

dataframe written from 0 to 200
dataframe written from 201 to 401
dataframe written from 402 to 602
dataframe written from 603 to 803
dataframe written from 804 to 1004
dataframe written from 1005 to 1205
dataframe written from 1206 to 1406
dataframe written from 1407 to 1607
dataframe written from 1608 to 1808
dataframe written from 1809 to 2009
dataframe written from 2010 to 2210
dataframe written from 2211 to 2411
dataframe written from 2412 to 2612
dataframe written from 2613 to 2813
dataframe written from 2814 to 3014
dataframe written from 3015 to 3215
dataframe written from 3216 to 3416
dataframe written from 3417 to 3617
dataframe written from 3618 to 3818
dataframe written from 3819 to 4019
dataframe written from 4020 to 4220
dataframe written from 4221 to 4421
dataframe written from 4422 to 4622
dataframe written from 4623 to 4823
dataframe written from 4824 to 5024
dataframe written from 5025 to 5225
dataframe written from 5226 to 5426
dataframe written from 5427 to 5627
dat

KeyboardInterrupt: 